<a href="https://colab.research.google.com/github/BF667/RVC/blob/main/RVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title <big> ➡️ **INSTALL**

import torch
from IPython.display import clear_output
from ipywidgets import Button


git = "https://github.com/BF667/RVC"
dir = "/co" + "nte" + "nt/R" + "V" + "C"

!git clone $git $dir &> /dev/null
%cd $dir

!pip install --no-cache-dir -qq uv
!uv pip install --no-cache-dir -qq -r requirements.txt


clear_output()
Button(description="\u2714 Insalling done !", button_style="success")

In [ ]:

#@title <big>🔎 **Model Loading**

%cd /content/RVC

import os
import subprocess
import ipywidgets as widgets
from IPython.display import display, clear_output


# Creating widgets for entering URL and directory name
url_input = widgets.Text(
    value='',
    placeholder='Enter the URL of the zip file',
    description='URL:',
    layout=widgets.Layout(width='50%')
)
dir_name_input = widgets.Text(
    value='',
    placeholder='Enter the directory name for the model',
    description='Model name:',
    layout=widgets.Layout(width='50%')
)
output = widgets.Output(layout=widgets.Layout(width='50%'))


def on_button_click(b):
    with output:
        output.clear_output()
        url = url_input.value
        dir_name = dir_name_input.value

        try:
            # Execute the script via subprocess with arguments
            result = subprocess.run(['python3', '-m', "rvc.modules.model_manager", url, dir_name], capture_output=True, text=True)
            if result.returncode == 0:
                print(result.stdout)
                clear_output()
                list_installed_models()
            else:
                print(f"Error: {result.stderr}")
        except Exception as e:
            print(f'Error: {e}')


download_button = widgets.Button(
    description='Download and unpack',
    disabled=False,
    button_style='',
    tooltip='Click to download and unpack the model',
    icon='download',
    layout=widgets.Layout(width='30%', height='50px')
)
download_button.on_click(on_button_click)

# Display widgets
display(url_input)
display(dir_name_input)
display(download_button)
display(output)


# Function to display the list of installed models
def list_installed_models():
    rvc_models_dir = os.path.join(os.getcwd(), "models", "RVC_models")
    print("\033[1;32m\nInstalled models:\n\033[0m")
    models = [d for d in os.listdir(rvc_models_dir) if os.path.isdir(os.path.join(rvc_models_dir, d))]
    for model in models:
        print("\033[1;92m", model, "\033[0m")
    print('\nCopy the model name and paste it into the "model_name" field.')

In [ ]:
#@title Run RVC


%cd /content/RVC

import os
from IPython.display import display, Audio

#@markdown ---
#@markdown * **Model name:**
model_name = "" # @param {"type":"string","placeholder":"Enter the model name you specified when loading the model"}
#@markdown * **Path to the audio file:**
input_path = "" # @param {"type":"string","placeholder":"Enter the path to the audio file"}
#@markdown ---
#@markdown * **Voice pitch (`-24` Male voice || Female voice `24`):**
pitch = 0 #@param {type:"slider", min:-24, max:24, step:0.5}
#@markdown * **Pitch extraction method:**
method = "rmvpe"  # @param ["rmvpe", "fcpe", "crepe", "crepe-tiny"]
#@markdown ---
index_rate = 0.25  # @param {type:"slider", min:0, max:1, step:0.01}
rms = 1  # @param {type:"slider", min:0, max:1, step:0.01}
protect = 0.33  # @param {type:"slider", min:0, max:0.5, step:0.01}
f0_min = 50  # @param {type:"slider", min:1, max:100, step:1}
f0_max = 1100  # @param {type:"slider", min:400, max:16000, step:10}
#@markdown ---
#@markdown * **Output file format:**
format = "mp3"  # @param ["wav", "flac", "mp3", "ogg", "opus", "m4a", "aiff", "ac3"]

try:
    !python3 -m rvc.infer.infer_cli rvc \
                                    --input_path $input_path \
                                    --rvc_model $model_name \
                                    --f0_method $method \
                                    --f0_min $f0_min \
                                    --f0_max $f0_max \
                                    --hop_length 32 \
                                    --rvc_pitch $pitch \
                                    --protect $protect \
                                    --index_rate $index_rate \
                                    --volume_envelope $rms \
                                    --output_format $format

    base_name = os.path.splitext(os.path.basename(input_path))[0]
    if len(base_name) > 50:
        base_name = "Made_with_RVC"
    output_path = os.path.join(
        os.getcwd(), "output", "RVC_output",
        f"{base_name}_({model_name}).{format}",
        )
    display(Audio(output_path, rate=44100))
except Exception as e:
    print(f"Error: {e}")